<a href="https://colab.research.google.com/github/Himagination/TensorFlow_Developer/blob/main/07_NLP_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Typical Architecture of Recurrent Neural Network.

* Input text(s): Target texts/sequence you'd like to discover patterns in
* Input Layer: Takes in target sequence
* Text Vectorization layer: Maps Input Sequences to numbers.
* Embedding: Turns mapping of text vectors to embedding matrix(representation of how words relate)
* RNN Cell(s): Find patterns in sequences.
* Hidden Activation: Adds non-linearity to learned features(non-straight line)
* Pooling Layer: Reduces the dimensionality of learned sequence features(usually for Conv1D modules)
* Fully Connected Layer: Further refines learned features from recurrent layers.
* Output layer: Takes Learned features and outputs them in shape of target labels.
* Output activation: Adds non-linearities to output layer.

# Introduction to NLP fundamentals

NLP has the goal of deriving information out of natural language(could be sequences text or speech)

Another common term for NLP problems is sequences problems(seq2seq)

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-2d1b3150-5eb1-65ae-b3ea-5c4c9f533eaa)


## Writing Helper Functions

- unzip_data: Unzips filename into the current working directory
- create_tensorboard_callback: Creates a TensorBoard callback instead to store log files
- plot_loss_curves: Returns Separate Loss Curves for training and Validation metrics
- compare_historys: Compares two TensorFlow model History Objects

### unzip_data

In [4]:
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory

  Args:
    filename(str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

### create_tensorboard_callback

In [2]:
import datetime
import tensorflow as tf

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instance to store log files

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

### plot_loss_curves

In [3]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns Separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object
  """

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title("Loss")
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

## Get a dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset(Text sample of Tweets labelled as disaster or not disaster).

url: https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-01-21 18:53:42--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.128, 173.194.215.128, 108.177.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-01-21 18:53:42 (153 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Becoming one with data

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
train_df['text'][2]

"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"

In [10]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [11]:
# What does the test dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [12]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
# How many total sample?
len(train_df), len(test_df)

(7613, 3263)

In [14]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df) - 5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Strikers in Phalaborwa striking over the CHINESE taking their jobs. Strikers burning buildings attacking cars 2... http://t.co/08LnGClZsj

---

Target: 1 (real disaster)
Text:
#News: 'Many deaths' in shipwreck: Rescuers are trying to save hundreds of migrants after the... http://t.co/tX51oYbrN6 via @TheNewsHype

---

Target: 0 (not real disaster)
Text:
matako_3: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass.

---

Target: 0 (not real disaster)
Text:
@thoutaylorbrown I feel like accidents are just drawn to you but I'm happy you've survived all of them. Hope you're okay!!!

---

Target: 0 (not real disaster)
Text:
@XGN_Infinity @Ronin_Carbon HAHAH Mutual host preset Bal no nades radar on = demolish

---



### Split data into training and validation sets

In [15]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), 
                                                                            train_df_shuffled['target'].to_numpy(), 
                                                                            test_size=0.1, 
                                                                            random_state=42)

In [16]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_sentences)

(6851, 6851, 762, 762)

In [17]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with text problem, one of the first things to do is to convert texts to numbers.

There are few ways to do this, namely:
* Tokenization - direct mapping of token(a token could be a word or a character) to number.
* Embedding - create a matrix of feature vector for each token(the size of the feature vector can be defined and this embedding can be learned)


### Text Vectorization(tokenization)

In [22]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int", 
                                    output_sequence_length=None)

In [25]:
# Find the average number of tokens(words) in the training tweets
round((sum([len(i.split()) for i in train_sentences])) / len(train_sentences))

15

In [26]:
# Setup text vectorization variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, 
                                    output_mode="int", 
                                    output_sequence_length=max_length)

In [27]:
# Fit the text vectorizer to the training data
text_vectorizer.adapt(train_sentences)

In [28]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [29]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\ {random_sentence}\
\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:\ #CDCwhistleblower 2 deaths from measles complications in 10 yrs everyone looses their shit. 8 dead from Legionnaires in a month &amp; crickets

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,   70,  401,   20, 5149, 5943,    4,  828, 1914,  238, 5191,
         131,  235,  589,  115]])>

In [31]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 Most common words: {top_5_words}")
print(f"5 Least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 Most common words: ['', '[UNK]', 'the', 'a', 'in']
5 Least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Create an Embedding using an Embedding Layer.

To make our embedding, we're going to use TensorFlow embedding layer.

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for eg a value of 100 would mean each token gets represented by a vector of length 100
* `input_length` = length of the sequences being passed to the embedding layer

In [32]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, 
                             output_dim=128, 
                             input_length=max_length)
embedding

In [36]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
\n\nEmbedded version:")

# Embed the random sentence(turn it into dense vectors)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @LifeAintFairKid if I did I'd smoke you up brooo!

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01313612,  0.02074124,  0.03961201, ...,  0.0195921 ,
          0.00978654, -0.02534814],
        [ 0.02684397, -0.0263682 , -0.04967499, ...,  0.01043807,
         -0.02980238, -0.03799772],
        [ 0.03150307,  0.04026164,  0.02827488, ..., -0.02130791,
         -0.04541652,  0.04636134],
        ...,
        [-0.03905207, -0.00688601, -0.02104526, ..., -0.00652734,
         -0.02840608,  0.02519   ],
        [-0.03905207, -0.00688601, -0.02104526, ..., -0.00652734,
         -0.02840608,  0.02519   ],
        [-0.03905207, -0.00688601, -0.02104526, ..., -0.00652734,
         -0.02840608,  0.02519   ]]], dtype=float32)>

## Modelling a text dataset and running series of experiments.

* Experiment 0: Naive Bayes with TF-IDF encoder(Baseline)
* Experiment 1: Feed-Forward Neural Network(Dense Model)
* Experiment 2: LSTM(RNN)
* Experiment 3: GRU(RNN)
* Experiment 4: Bidirectional-LSTM(RNN)
* Experiment 5: 1D Convolutional Neural Network
* Experiment 6: TensorFlow Hub Pretrained Feature Extractor
* Experiment 7: TensorFlow Hub Pretrained Feature Extractor(10% of data)

Use the standard steps in modelling with TensorFlow
* Create a model
* Build a model
* Fit a model
* Evaluate our model

### Model 0: Getting a baseline

As with all Machine Learning modelling experiments, it's important to create a baseline model so we've got a benchmark for future experiment to build upon.

To create our baseline, we'll use Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert  our words to numbers.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [39]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [40]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

Let's create function to compare our model's prediction with the truth labels using the following:
* Accuracy
* Precision
* Recall
* F1-Score

In [44]:
# Function to evaluate: accuracy, prediction, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
  model_results = {"accuracy": model_accuracy, 
                   "precision": model_precision, 
                   "recall": model_recall, 
                   "f1": model_f1}
  return model_results

In [45]:
# Get baseline results
baseline_results = calculate_results(y_true=val_labels, 
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}